In [1]:
import requests
import time
from datetime import datetime
import pandas as pd
import os
import json
import uuid
import sqlite3
import sys
from dotenv import load_dotenv
# Load the .env file
load_dotenv()
# Access the environment variables
project_folder_path = os.getenv('PROJECT_FOLDER_PATH')
print(project_folder_path)
project_folder_path = '/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-database'
sys.path.append(project_folder_path)
from utils.db_functs import *
from utils.requests_functs import *
from utils.html_functs import *
from utils.pdf_functs import *
from utils.utils_functs import *

/home/roruizf/anaconda3/envs/data_env/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/roruizf/anaconda3/envs/data_env/lib/python3.9/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-database


### Database and directories

In [2]:
db_file_name = 'cev-database-coordinates-v1.db'
db_file_path = os.path.join(project_folder_path, 'data', 'sqlite', db_file_name)
csv_file_path = '/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-database-reports/pipeline/CEV-Chile-2022-September-ratings.csv'

## 1) Define parameters to run a `search`
This can be a new search or retake a previous one

In [3]:
df = pd.read_csv(csv_file_path)
df.head()

,Identificación Vivienda,Proyecto,Tipología,Región,Comuna,Status,CE,CEE,Hash Id
0,64010-Dpto. 1 Block 1 fachada Oeste,Conjunto Habitacional Cariquima,Departamento,Tarapacá,Iquique,Pre-calificación,E,E,75f9b27842668f5c0b297be40ed81233b0c32e5d
1,64013-Dpto. 4 Block 1 fachada Oeste,Conjunto Habitacional Cariquima,Departamento,Tarapacá,Iquique,Pre-calificación,E,E,95e25e05f862e74a5a6c47388a9ee42ad3c05b65
2,64021-Dpto. -1 Block 2 fachada Oeste,Conjunto Habitacional Cariquima,Departamento,Tarapacá,Iquique,Pre-calificación,E,E,920825d249c12e45b160baf1ee74f22294a74bbe
3,64024-Dpto.3 Block 2 fachada Oeste,Conjunto Habitacional Cariquima,Departamento,Tarapacá,Iquique,Pre-calificación,E,E,6829a02bbd311bbb8e03965166d792dbb2a03bc5
4,64027-Dpto. 1 Block 2 fachada Este,Conjunto Habitacional Cariquima,Departamento,Tarapacá,Iquique,Pre-calificación,E,E,517a45fcb0f3d84e2beb479253abaad30899037c


Set additional features to track the status of downloads as well as times and file names

In [4]:
# Add additional columns
query = f"""
        SELECT * FROM comunas;
        """
print(query)
comunas_df = create_dataframe_from_query(db_file_path, query)
#comunas_df['comuna_id'] = comunas_df['comuna_id'].astype(str)

df['comuna_id'] = df['Comuna'].map(comunas_df.set_index('comuna_name')['comuna_id'].to_dict())
df['region_id'] = df['Comuna'].map(comunas_df.set_index('comuna_name')['region_id'].to_dict())
df['tipo_evaluacion'] = df['Status'].map({'Pre-calificación':1,
                                          'Calificación':2
                                            })

df.head()


        SELECT * FROM comunas;
        


,Identificación Vivienda,Proyecto,Tipología,Región,Comuna,Status,CE,CEE,Hash Id,comuna_id,region_id,tipo_evaluacion
0,64010-Dpto. 1 Block 1 fachada Oeste,Conjunto Habitacional Cariquima,Departamento,Tarapacá,Iquique,Pre-calificación,E,E,75f9b27842668f5c0b297be40ed81233b0c32e5d,113,1,1
1,64013-Dpto. 4 Block 1 fachada Oeste,Conjunto Habitacional Cariquima,Departamento,Tarapacá,Iquique,Pre-calificación,E,E,95e25e05f862e74a5a6c47388a9ee42ad3c05b65,113,1,1
2,64021-Dpto. -1 Block 2 fachada Oeste,Conjunto Habitacional Cariquima,Departamento,Tarapacá,Iquique,Pre-calificación,E,E,920825d249c12e45b160baf1ee74f22294a74bbe,113,1,1
3,64024-Dpto.3 Block 2 fachada Oeste,Conjunto Habitacional Cariquima,Departamento,Tarapacá,Iquique,Pre-calificación,E,E,6829a02bbd311bbb8e03965166d792dbb2a03bc5,113,1,1
4,64027-Dpto. 1 Block 2 fachada Este,Conjunto Habitacional Cariquima,Departamento,Tarapacá,Iquique,Pre-calificación,E,E,517a45fcb0f3d84e2beb479253abaad30899037c,113,1,1


In [5]:
# Evaluacion id: generated from 4 columns with fcuntion string_to_uuid
df['eval_id'] = (df['comuna_id'].astype(str) + '_' + df['region_id'].astype(str) + '_' + df['tipo_evaluacion'].astype(str) + '_' + df['Identificación Vivienda'].astype(str)).apply(string_to_uuid)
df['eval_id'] = df['eval_id'].astype(str)

In [6]:
df.head()

,Identificación Vivienda,Proyecto,Tipología,Región,Comuna,Status,CE,CEE,Hash Id,comuna_id,region_id,tipo_evaluacion,eval_id
0,64010-Dpto. 1 Block 1 fachada Oeste,Conjunto Habitacional Cariquima,Departamento,Tarapacá,Iquique,Pre-calificación,E,E,75f9b27842668f5c0b297be40ed81233b0c32e5d,113,1,1,6ca5fbbe-52cb-5ba5-90bd-b31946e757c7
1,64013-Dpto. 4 Block 1 fachada Oeste,Conjunto Habitacional Cariquima,Departamento,Tarapacá,Iquique,Pre-calificación,E,E,95e25e05f862e74a5a6c47388a9ee42ad3c05b65,113,1,1,3786a536-3cbf-51ab-8af7-f78d48862bcb
2,64021-Dpto. -1 Block 2 fachada Oeste,Conjunto Habitacional Cariquima,Departamento,Tarapacá,Iquique,Pre-calificación,E,E,920825d249c12e45b160baf1ee74f22294a74bbe,113,1,1,cd0f9119-30b6-5329-8a60-e12c3b83781a
3,64024-Dpto.3 Block 2 fachada Oeste,Conjunto Habitacional Cariquima,Departamento,Tarapacá,Iquique,Pre-calificación,E,E,6829a02bbd311bbb8e03965166d792dbb2a03bc5,113,1,1,aa9e54e5-6cb0-566a-b67f-18a619f13ea8
4,64027-Dpto. 1 Block 2 fachada Este,Conjunto Habitacional Cariquima,Departamento,Tarapacá,Iquique,Pre-calificación,E,E,517a45fcb0f3d84e2beb479253abaad30899037c,113,1,1,180693d6-0810-51d6-a8a2-06658a712963


In [7]:
df['old_file_name'] = df['region_id'].astype(str) + '_' + df['comuna_id'].astype(str) + '_' + df['tipo_evaluacion'].astype(str) + '_' + df['Hash Id'].astype(str) + '.pdf'
df['new_file_name'] = df['region_id'].astype(str) + '_' + df['comuna_id'].astype(str) + '_' + df['tipo_evaluacion'].astype(str) + '_' + df['eval_id'].astype(str) + '.pdf'

In [8]:
df['status'] = None
df['pdf_filename'] = None
df['downloaded_at'] = None
df.head()

,Identificación Vivienda,Proyecto,Tipología,Región,Comuna,Status,CE,CEE,Hash Id,comuna_id,region_id,tipo_evaluacion,eval_id,old_file_name,new_file_name,status,pdf_filename,downloaded_at
0,64010-Dpto. 1 Block 1 fachada Oeste,Conjunto Habitacional Cariquima,Departamento,Tarapacá,Iquique,Pre-calificación,E,E,75f9b27842668f5c0b297be40ed81233b0c32e5d,113,1,1,6ca5fbbe-52cb-5ba5-90bd-b31946e757c7,1_113_1_75f9b27842668f5c0b297be40ed81233b0c32e...,1_113_1_6ca5fbbe-52cb-5ba5-90bd-b31946e757c7.pdf,None,None,None
1,64013-Dpto. 4 Block 1 fachada Oeste,Conjunto Habitacional Cariquima,Departamento,Tarapacá,Iquique,Pre-calificación,E,E,95e25e05f862e74a5a6c47388a9ee42ad3c05b65,113,1,1,3786a536-3cbf-51ab-8af7-f78d48862bcb,1_113_1_95e25e05f862e74a5a6c47388a9ee42ad3c05b...,1_113_1_3786a536-3cbf-51ab-8af7-f78d48862bcb.pdf,None,None,None
2,64021-Dpto. -1 Block 2 fachada Oeste,Conjunto Habitacional Cariquima,Departamento,Tarapacá,Iquique,Pre-calificación,E,E,920825d249c12e45b160baf1ee74f22294a74bbe,113,1,1,cd0f9119-30b6-5329-8a60-e12c3b83781a,1_113_1_920825d249c12e45b160baf1ee74f22294a74b...,1_113_1_cd0f9119-30b6-5329-8a60-e12c3b83781a.pdf,None,None,None
3,64024-Dpto.3 Block 2 fachada Oeste,Conjunto Habitacional Cariquima,Departamento,Tarapacá,Iquique,Pre-calificación,E,E,6829a02bbd311bbb8e03965166d792dbb2a03bc5,113,1,1,aa9e54e5-6cb0-566a-b67f-18a619f13ea8,1_113_1_6829a02bbd311bbb8e03965166d792dbb2a03b...,1_113_1_aa9e54e5-6cb0-566a-b67f-18a619f13ea8.pdf,None,None,None
4,64027-Dpto. 1 Block 2 fachada Este,Conjunto Habitacional Cariquima,Departamento,Tarapacá,Iquique,Pre-calificación,E,E,517a45fcb0f3d84e2beb479253abaad30899037c,113,1,1,180693d6-0810-51d6-a8a2-06658a712963,1_113_1_517a45fcb0f3d84e2beb479253abaad3089903...,1_113_1_180693d6-0810-51d6-a8a2-06658a712963.pdf,None,None,None


## 3) Check first if any pdf files has been downloaded
This is useful in case a seach has been run and has not completed

### 3.1) Read all html files for the corresponding `search_date` and `search_id`

In [9]:
# Define the directory path
pdf_files_dir = '/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-database-reports/data'

# Check if the directory exists
if not os.path.exists(pdf_files_dir):
    # If it doesn't exist, create the directory
    os.makedirs(pdf_files_dir)
    print(f"Directory '{pdf_files_dir}' created.")

# List all files in the directory
pdf_file_paths = find_pdf_files(pdf_files_dir)
print(f'The directory: {pdf_files_dir} contains {len(pdf_file_paths)} out of {df.shape[0]}')

The directory: /mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-database-reports/data contains 12351 out of 99852


### 3.2) Fill on dataframe all those rows corresponding to files already downloaded

In [10]:
count = 0
for pdf_file_path in pdf_file_paths:
    print(f'file {count + 1} out of {len(pdf_file_paths)}')
    pdf_file_name = os.path.split(pdf_file_path)[-1]
    pdf_file_list = pdf_file_name.split('_')
    region_id = int(pdf_file_list[0])
    comuna_id = int(pdf_file_list[1])
    tipo_evaluacion = int(pdf_file_list[2])
    hash_id = pdf_file_list[-1].split('.')[0]
    row = df[df['region_id'].astype(str) + '_' + df['comuna_id'].astype(str) + '_' + df['tipo_evaluacion'].astype(str)+ '_' + df['Hash Id'].astype(str) == str(region_id) + '_' + str(comuna_id) + '_' + str(tipo_evaluacion) + '_' + str(hash_id)]
    if not row.empty:
        df.loc[row.index, 'status'] = 'Successful'
        df.loc[row.index, 'pdf_filename'] = pdf_file_name
        df.loc[row.index, 'pdf_file_path'] = pdf_file_path
        #df.loc[row.index, 'downloaded_at'] = None    
    count = count + 1

file 1 out of 12351
file 2 out of 12351
file 3 out of 12351
file 4 out of 12351
file 5 out of 12351
file 6 out of 12351
file 7 out of 12351
file 8 out of 12351
file 9 out of 12351
file 10 out of 12351
file 11 out of 12351
file 12 out of 12351
file 13 out of 12351
file 14 out of 12351
file 15 out of 12351
file 16 out of 12351
file 17 out of 12351
file 18 out of 12351
file 19 out of 12351
file 20 out of 12351
file 21 out of 12351
file 22 out of 12351
file 23 out of 12351
file 24 out of 12351
file 25 out of 12351
file 26 out of 12351
file 27 out of 12351
file 28 out of 12351
file 29 out of 12351
file 30 out of 12351
file 31 out of 12351
file 32 out of 12351
file 33 out of 12351
file 34 out of 12351
file 35 out of 12351
file 36 out of 12351
file 37 out of 12351
file 38 out of 12351
file 39 out of 12351
file 40 out of 12351
file 41 out of 12351
file 42 out of 12351
file 43 out of 12351
file 44 out of 12351
file 45 out of 12351
file 46 out of 12351
file 47 out of 12351
file 48 out of 12351
f

file 380 out of 12351
file 381 out of 12351
file 382 out of 12351
file 383 out of 12351
file 384 out of 12351
file 385 out of 12351
file 386 out of 12351
file 387 out of 12351
file 388 out of 12351
file 389 out of 12351
file 390 out of 12351
file 391 out of 12351
file 392 out of 12351
file 393 out of 12351
file 394 out of 12351
file 395 out of 12351
file 396 out of 12351
file 397 out of 12351
file 398 out of 12351
file 399 out of 12351
file 400 out of 12351
file 401 out of 12351
file 402 out of 12351
file 403 out of 12351
file 404 out of 12351
file 405 out of 12351
file 406 out of 12351
file 407 out of 12351
file 408 out of 12351
file 409 out of 12351
file 410 out of 12351
file 411 out of 12351
file 412 out of 12351
file 413 out of 12351
file 414 out of 12351
file 415 out of 12351
file 416 out of 12351
file 417 out of 12351
file 418 out of 12351
file 419 out of 12351
file 420 out of 12351
file 421 out of 12351
file 422 out of 12351
file 423 out of 12351
file 424 out of 12351
file 425 o

file 754 out of 12351
file 755 out of 12351
file 756 out of 12351
file 757 out of 12351
file 758 out of 12351
file 759 out of 12351
file 760 out of 12351
file 761 out of 12351
file 762 out of 12351
file 763 out of 12351
file 764 out of 12351
file 765 out of 12351
file 766 out of 12351
file 767 out of 12351
file 768 out of 12351
file 769 out of 12351
file 770 out of 12351
file 771 out of 12351
file 772 out of 12351
file 773 out of 12351
file 774 out of 12351
file 775 out of 12351
file 776 out of 12351
file 777 out of 12351
file 778 out of 12351
file 779 out of 12351
file 780 out of 12351
file 781 out of 12351
file 782 out of 12351
file 783 out of 12351
file 784 out of 12351
file 785 out of 12351
file 786 out of 12351
file 787 out of 12351
file 788 out of 12351
file 789 out of 12351
file 790 out of 12351
file 791 out of 12351
file 792 out of 12351
file 793 out of 12351
file 794 out of 12351
file 795 out of 12351
file 796 out of 12351
file 797 out of 12351
file 798 out of 12351
file 799 o

file 1123 out of 12351
file 1124 out of 12351
file 1125 out of 12351
file 1126 out of 12351
file 1127 out of 12351
file 1128 out of 12351
file 1129 out of 12351
file 1130 out of 12351
file 1131 out of 12351
file 1132 out of 12351
file 1133 out of 12351
file 1134 out of 12351
file 1135 out of 12351
file 1136 out of 12351
file 1137 out of 12351
file 1138 out of 12351
file 1139 out of 12351
file 1140 out of 12351
file 1141 out of 12351
file 1142 out of 12351
file 1143 out of 12351
file 1144 out of 12351
file 1145 out of 12351
file 1146 out of 12351
file 1147 out of 12351
file 1148 out of 12351
file 1149 out of 12351
file 1150 out of 12351
file 1151 out of 12351
file 1152 out of 12351
file 1153 out of 12351
file 1154 out of 12351
file 1155 out of 12351
file 1156 out of 12351
file 1157 out of 12351
file 1158 out of 12351
file 1159 out of 12351
file 1160 out of 12351
file 1161 out of 12351
file 1162 out of 12351
file 1163 out of 12351
file 1164 out of 12351
file 1165 out of 12351
file 1166 o

file 1480 out of 12351
file 1481 out of 12351
file 1482 out of 12351
file 1483 out of 12351
file 1484 out of 12351
file 1485 out of 12351
file 1486 out of 12351
file 1487 out of 12351
file 1488 out of 12351
file 1489 out of 12351
file 1490 out of 12351
file 1491 out of 12351
file 1492 out of 12351
file 1493 out of 12351
file 1494 out of 12351
file 1495 out of 12351
file 1496 out of 12351
file 1497 out of 12351
file 1498 out of 12351
file 1499 out of 12351
file 1500 out of 12351
file 1501 out of 12351
file 1502 out of 12351
file 1503 out of 12351
file 1504 out of 12351
file 1505 out of 12351
file 1506 out of 12351
file 1507 out of 12351
file 1508 out of 12351
file 1509 out of 12351
file 1510 out of 12351
file 1511 out of 12351
file 1512 out of 12351
file 1513 out of 12351
file 1514 out of 12351
file 1515 out of 12351
file 1516 out of 12351
file 1517 out of 12351
file 1518 out of 12351
file 1519 out of 12351
file 1520 out of 12351
file 1521 out of 12351
file 1522 out of 12351
file 1523 o

...

file 10745 out of 12351
file 10746 out of 12351
file 10747 out of 12351
file 10748 out of 12351
file 10749 out of 12351
file 10750 out of 12351
file 10751 out of 12351
file 10752 out of 12351
file 10753 out of 12351
file 10754 out of 12351
file 10755 out of 12351
file 10756 out of 12351
file 10757 out of 12351
file 10758 out of 12351
file 10759 out of 12351
file 10760 out of 12351
file 10761 out of 12351
file 10762 out of 12351
file 10763 out of 12351
file 10764 out of 12351
file 10765 out of 12351
file 10766 out of 12351
file 10767 out of 12351
file 10768 out of 12351
file 10769 out of 12351
file 10770 out of 12351
file 10771 out of 12351
file 10772 out of 12351
file 10773 out of 12351
file 10774 out of 12351
file 10775 out of 12351
file 10776 out of 12351
file 10777 out of 12351
file 10778 out of 12351
file 10779 out of 12351
file 10780 out of 12351
file 10781 out of 12351
file 10782 out of 12351
file 10783 out of 12351
file 10784 out of 12351
file 10785 out of 12351
file 10786 out o

file 11087 out of 12351
file 11088 out of 12351
file 11089 out of 12351
file 11090 out of 12351
file 11091 out of 12351
file 11092 out of 12351
file 11093 out of 12351
file 11094 out of 12351
file 11095 out of 12351
file 11096 out of 12351
file 11097 out of 12351
file 11098 out of 12351
file 11099 out of 12351
file 11100 out of 12351
file 11101 out of 12351
file 11102 out of 12351
file 11103 out of 12351
file 11104 out of 12351
file 11105 out of 12351
file 11106 out of 12351
file 11107 out of 12351
file 11108 out of 12351
file 11109 out of 12351
file 11110 out of 12351
file 11111 out of 12351
file 11112 out of 12351
file 11113 out of 12351
file 11114 out of 12351
file 11115 out of 12351
file 11116 out of 12351
file 11117 out of 12351
file 11118 out of 12351
file 11119 out of 12351
file 11120 out of 12351
file 11121 out of 12351
file 11122 out of 12351
file 11123 out of 12351
file 11124 out of 12351
file 11125 out of 12351
file 11126 out of 12351
file 11127 out of 12351
file 11128 out o

file 11430 out of 12351
file 11431 out of 12351
file 11432 out of 12351
file 11433 out of 12351
file 11434 out of 12351
file 11435 out of 12351
file 11436 out of 12351
file 11437 out of 12351
file 11438 out of 12351
file 11439 out of 12351
file 11440 out of 12351
file 11441 out of 12351
file 11442 out of 12351
file 11443 out of 12351
file 11444 out of 12351
file 11445 out of 12351
file 11446 out of 12351
file 11447 out of 12351
file 11448 out of 12351
file 11449 out of 12351
file 11450 out of 12351
file 11451 out of 12351
file 11452 out of 12351
file 11453 out of 12351
file 11454 out of 12351
file 11455 out of 12351
file 11456 out of 12351
file 11457 out of 12351
file 11458 out of 12351
file 11459 out of 12351
file 11460 out of 12351
file 11461 out of 12351
file 11462 out of 12351
file 11463 out of 12351
file 11464 out of 12351
file 11465 out of 12351
file 11466 out of 12351
file 11467 out of 12351
file 11468 out of 12351
file 11469 out of 12351
file 11470 out of 12351
file 11471 out o

file 11774 out of 12351
file 11775 out of 12351
file 11776 out of 12351
file 11777 out of 12351
file 11778 out of 12351
file 11779 out of 12351
file 11780 out of 12351
file 11781 out of 12351
file 11782 out of 12351
file 11783 out of 12351
file 11784 out of 12351
file 11785 out of 12351
file 11786 out of 12351
file 11787 out of 12351
file 11788 out of 12351
file 11789 out of 12351
file 11790 out of 12351
file 11791 out of 12351
file 11792 out of 12351
file 11793 out of 12351
file 11794 out of 12351
file 11795 out of 12351
file 11796 out of 12351
file 11797 out of 12351
file 11798 out of 12351
file 11799 out of 12351
file 11800 out of 12351
file 11801 out of 12351
file 11802 out of 12351
file 11803 out of 12351
file 11804 out of 12351
file 11805 out of 12351
file 11806 out of 12351
file 11807 out of 12351
file 11808 out of 12351
file 11809 out of 12351
file 11810 out of 12351
file 11811 out of 12351
file 11812 out of 12351
file 11813 out of 12351
file 11814 out of 12351
file 11815 out o

file 12118 out of 12351
file 12119 out of 12351
file 12120 out of 12351
file 12121 out of 12351
file 12122 out of 12351
file 12123 out of 12351
file 12124 out of 12351
file 12125 out of 12351
file 12126 out of 12351
file 12127 out of 12351
file 12128 out of 12351
file 12129 out of 12351
file 12130 out of 12351
file 12131 out of 12351
file 12132 out of 12351
file 12133 out of 12351
file 12134 out of 12351
file 12135 out of 12351
file 12136 out of 12351
file 12137 out of 12351
file 12138 out of 12351
file 12139 out of 12351
file 12140 out of 12351
file 12141 out of 12351
file 12142 out of 12351
file 12143 out of 12351
file 12144 out of 12351
file 12145 out of 12351
file 12146 out of 12351
file 12147 out of 12351
file 12148 out of 12351
file 12149 out of 12351
file 12150 out of 12351
file 12151 out of 12351
file 12152 out of 12351
file 12153 out of 12351
file 12154 out of 12351
file 12155 out of 12351
file 12156 out of 12351
file 12157 out of 12351
file 12158 out of 12351
file 12159 out o

In [11]:
df[df['status'] == 'Successful'][['pdf_file_path']].reset_index(drop=True).iloc[0].values

KeyError: "None of [Index(['pdf_file_path'], dtype='object')] are in the [columns]"

In [ ]:
df['pdf_filename'].isna().value_counts(dropna=False)

In [ ]:
new_names_df = df[df['status'] == 'Successful'].reset_index(drop=True).copy()
new_names_df['rename_status'] = None
new_names_df

for index, row in new_names_df.iterrows():
    print(f'file {index + 1} out of {new_names_df.shape[0]}')
    new_pdf_file_path = os.path.join(os.path.split(pdf_file_path)[0],  row['new_file_name'])
    os.rename(row['pdf_file_path'], new_pdf_file_path)
    #new_names_df.loc[row.index, 'rename_status'] = 'renamed'
    

In [ ]:
new_names_df.head()

In [ ]:
new_pdf_file_path